In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import io
import requests
import os

from scipy.stats import zscore
from sklearn import preprocessing, metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv("../../../Datasets/Flows/train.csv")

# Display the DataFrame
display(df)

,service,traffic,total_bytes,total_pkts,pkt_difference,byte_difference,total_data_pkts,payload_ratio,total_payload_volume,fwd_bwd_pkts_diff,...,history_originator,history_responder,proto_1,proto_2,proto_3,pkts_unidirectional_traffic_0,pkts_unidirectional_traffic_1,iat_is_unidirectional_False,iat_is_unidirectional_True,is_attack
0,0,rudeadyet,-0.004437,3,-0.083659,-0.003766,0,-0.558121,-0.420430,-0.068478,...,1,1,1,0,0,0,1,1,0,1
1,2,normal,-0.004434,10,0.034060,-0.003763,5,0.972797,0.954078,0.000492,...,1,1,1,0,0,0,1,1,0,0
2,0,normal,-0.004383,6,0.465697,-0.003696,6,2.503716,1.228979,0.000492,...,1,2,0,1,0,0,1,1,0,0
3,0,netscan,-0.004437,2,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,1,1,1,0,0,1,0,0,1,1
4,0,normal,-0.004437,6,-0.122899,-0.003766,6,2.503716,1.228979,0.000492,...,1,1,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439302,2,normal,-0.004434,10,0.073300,-0.003762,5,0.972797,0.954078,0.000492,...,1,1,1,0,0,0,1,1,0,0
439303,0,normal,-0.004437,10,-0.122899,-0.003766,5,0.972797,0.954078,0.000492,...,1,1,1,0,0,0,1,0,1,0
439304,2,normal,-0.004434,10,0.073300,-0.003762,5,0.972797,0.954078,0.000492,...,1,1,1,0,0,0,1,1,0,0
439305,0,camoverflow,-0.004437,1,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,0,0,1,0,0,0,1,0,1,1


In [3]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0    225000
1    214307
Name: is_attack, dtype: int64

In [4]:
df.groupby('traffic')['traffic'].count()

traffic
apachekiller      1756
arpspoofing       8361
camoverflow      46934
mqttmalaria       3709
netscan          46767
normal          225000
rudeadyet        54088
slowloris        47685
slowread          5007
Name: traffic, dtype: int64

In [5]:
# Read the CSV file into a DataFrame
df = pd.read_csv("../../../Datasets/Flows/test.csv")

# Display the DataFrame
display(df)

,service,traffic,total_bytes,total_pkts,pkt_difference,byte_difference,total_data_pkts,payload_ratio,total_payload_volume,fwd_bwd_pkts_diff,...,history_originator,history_responder,proto_1,proto_2,proto_3,pkts_unidirectional_traffic_0,pkts_unidirectional_traffic_1,iat_is_unidirectional_False,iat_is_unidirectional_True,is_attack
0,0,normal,-0.004383,6,0.465697,-0.003696,6,2.503716,1.228979,0.000492,...,1,2,0,1,0,0,1,1,0,0
1,2,normal,-0.004434,6,0.073300,-0.003762,6,2.503716,1.228979,0.000492,...,1,1,1,0,0,0,1,1,0,0
2,0,netscan,-0.004437,2,-0.122899,-0.003766,0,-0.558121,-0.420430,0.000492,...,1,1,1,0,0,1,0,0,1,1
3,2,normal,-0.004434,8,0.073300,-0.003762,4,0.972797,0.679176,0.000492,...,1,1,1,0,0,0,1,1,0,0
4,0,normal,-0.004383,10,0.465697,-0.003696,5,0.972797,0.954078,0.000492,...,1,2,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146431,0,rudeadyet,-0.004437,3,-0.083659,-0.003766,0,-0.558121,-0.420430,-0.018061,...,1,1,1,0,0,0,1,1,0,1
146432,0,slowloris,-0.004437,3,-0.083659,-0.003766,0,-0.558121,-0.420430,-0.526904,...,1,1,1,0,0,0,1,1,0,1
146433,0,normal,-0.004437,13,-0.122899,-0.003766,7,1.090560,1.503881,0.000492,...,1,1,1,0,0,0,1,0,1,0
146434,0,normal,-0.004437,10,-0.122899,-0.003766,5,0.972797,0.954078,0.000492,...,1,1,1,0,0,0,1,1,0,0


In [6]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0    75000
1    71436
Name: is_attack, dtype: int64

In [7]:
df.groupby('traffic')['traffic'].count()

traffic
apachekiller      578
arpspoofing      2861
camoverflow     15566
mqttmalaria      1252
netscan         15733
normal          75000
rudeadyet       17902
slowloris       15811
slowread         1733
Name: traffic, dtype: int64